<a href="https://colab.research.google.com/github/ahmedalaaseif/Bachelor/blob/master/data/manipulation/data_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wget

In [ ]:
%cd ".."

In [ ]:
%cd "content"

In [ ]:
%cd "drive"

In [ ]:
%cd "My Drive"

In [ ]:
%cd "Bachelor"

In [ ]:
%cd "labels"

In [ ]:
% ls

In [ ]:
import wget
import cv2
import requests

with open('/content/drive/My Drive/Bachelor/NSFW data/urls_sexy.txt', 'r') as myfile:

    lines = myfile.readlines()


print(lines[59].rstrip("\n"))
i=0
for line in lines:
    req = line.rstrip("\n")
    try:
        if(i>=0):
            print(req)
            wget.download(req)
            print("\n ---->"+ str((i/len(lines))*100)+" % \n")
    except:
        print("got it!!!")
    i+=1

In [ ]:
#### Testing if the written boudning box is working with 
%matplotlib inline
from matplotlib import pyplot as plt
import cv2

# Load in Grayscale
img = cv2.imread('/content/1 (1).jpeg', 0)


start_point = (273, 410)  
end_point = ( 273+126, 410+142) 
 
# Blue color in BGR 
color = (255, 0, 0) 
# Line thickness of 2 px 
thickness = 5
img = cv2.rectangle(img, start_point, end_point, color, thickness) 
plt.imshow(img)
plt.show()


In [ ]:
## Labeling Data
import requests
import json
import cv2
import os

## Path to your set of images you want to label
path = '/content/drive/My Drive/Bachelor/NSFW data'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
      if(file.endswith(".jpg") or file.endswith(".jpeg")):
        files.append(os.path.join(r, file))
i = 0
for file in files:
    if(i>=11322):
        r = requests.post(
"https://api.deepai.org/api/nsfw-detector",
    files={
        'image': open(str(file), 'rb'),
    },
    headers={'api-key': '147de7e2-0318-4be2-bca8-6791f9b3c206'}
)
        print(file)
        print("-------------> "+ str((i/len(files))*100)+" %")
        print("we finished " + str(i)+ "  images ")
        wjdata = r.json()
        im = cv2.imread(str(file))
        if(len(wjdata['output']["detections"])>0):
            toWrite = ""
            
            for c in wjdata['output']["detections"]:
                print("bounding box---> "+ str(c["bounding_box"]))
                boxWidth = c["bounding_box"][2]
                boxHeight = c["bounding_box"][3]
                centerX = (boxWidth/2) + c["bounding_box"][0]
                centerY = (boxHeight/2) + c["bounding_box"][1]
                toWrite+= "0 " + str(centerX/im.shape[1]) + " "+ str(centerY/im.shape[0])+ " "+ str(c["bounding_box"][2]/im.shape[1]) + " "+ str(c["bounding_box"][3]/im.shape[0]) +" \n"
            name = file


            
            if(name.endswith("jpg")):
              name = name[:-4]
            if(name.endswith("jpeg")):
              name = name[:-5]
            name += ".txt"
            file1 = open(name,"w" ) 
            file1.write(toWrite) 
            file1.close()
            print("NSFW")
        else:

            name = file
            if(name.endswith("jpg")):
              name = name[:-4]
            if(name.endswith("jpeg")):
              name = name[:-5]
            name += ".txt"
            file1 = open(name,"w") 
            file1.close()
            print("NOT NSFW")
    i+=1

    






In [ ]:
## Looking for duplicate names but different extensions
import os

## Path to your set of images you want to label
path = '/content/drive/My Drive/Bachelor/NSFW data'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
      if(file.endswith(".jpg") or file.endswith(".jpeg")):
        files.append(os.path.join(r, file))
filesTXT = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
      if(file.endswith(".txt")):
        filesTXT.append(os.path.join(r, file))
## The number of images is equal to the number of generated label files
print(len(files))
print(len(filesTXT))

In [ ]:
## Generating the needed train.txt file for YOLO
toWrite=""
for line in files:
  toWrite += line + "\n"

file1 = open("train.txt","w" ) 
file1.write(toWrite) 
file1.close()

In [ ]:
## Generating the needed obj.names file for YOLO
file1 = open("obj.names","w" ) 
file1.write("NSFW") 
file1.close()

In [ ]:
## Generating the needed obj.data file for YOLO
file1 = open("obj.data","w" ) 
file1.write("classes = 1 \ntrain = /content/darknet/train.txt \nvalid = /content/darknet/test.txt \nnames = /content/darknet/obj.names \nbackup= /content/drive/My\ drive/Bachelor/backup/ \nbackup = backup \nnot backup = backup/") 


file1.close()

In [ ]:
## Making sure that all the extensions are .jpg
for k in files:
  c= k
  if(k.endswith(".jpeg")):
    c = c[:-5] + ".jpg"
    if(c in files):
      print(c)
  

In [ ]:
## checking for missed images(not labeled)
notLabeled= []

for n in files:
  c = n
  if(c.endswith("jpg")):
    c = c[:-4]
  if(c.endswith("jpeg")):
    c = c[:-5]
  c = c + ".txt"
  if(c not in filesTXT):
    notLabeled.append(n)
print(notLabeled)
print(len(notLabeled))

In [ ]:
## Labeling the images with no label, not necessary if previous cell variable (notLabeled) is empty
import requests
import cv2
i = 0
for file in notLabeled:
    if(i>=0):
        r = requests.post(
"https://api.deepai.org/api/nsfw-detector",
    files={
        'image': open(str(file), 'rb'),
    },
    headers={'api-key': 'f01a1743-f34f-4ae8-93b3-a6086b4f125d'}
)
        print(file)
        print("-------------> "+ str((i/len(notLabeled))*100)+" %")
        print("we finished " + str(i)+ "  images ")
        wjdata = r.json()
        im = cv2.imread(str(file))
        if(len(wjdata['output']["detections"])>0):
            toWrite = ""
            
            for c in wjdata['output']["detections"]:
                print("bounding box---> "+ str(c["bounding_box"]))
                boxWidth = c["bounding_box"][2]
                boxHeight = c["bounding_box"][3]
                centerX = (boxWidth/2) + c["bounding_box"][0]
                centerY = (boxHeight/2) + c["bounding_box"][1]
                toWrite+= "0 " + str(centerX/im.shape[1]) + " "+ str(centerY/im.shape[0])+ " "+ str(c["bounding_box"][2]/im.shape[1]) + " "+ str(c["bounding_box"][3]/im.shape[0]) +" \n"
            name = file


            
            if(name.endswith("jpg")):
              name = name[:-4]
            if(name.endswith("jpeg")):
              name = name[:-5]
            name += ".txt"
            file1 = open(name,"w" ) 
            file1.write(toWrite) 
            file1.close()
            print("NSFW")
        else:
            name = file
            if(name.endswith("jpg")):
              name = name[:-4]
            if(name.endswith("jpeg")):
              name = name[:-5]
            name += ".txt"
            file1 = open(name,"w") 
            file1.close()
            print("NOT NSFW")
  
    i += 1